In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from Levenshtein import distance
from scipy import stats
from scipy.stats import mannwhitneyu
from Bio.Align import substitution_matrices  # Modern way to import matrices
import gzip
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
import os
import zipfile
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from scipy.stats import skew, kurtosis
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import export_graphviz
import pydotplus
from IPython.display import Image
import sys
import subprocess

In [ ]:
# Download combined_df from Google Drive Link
combined_df = pd.read_csv("combined_data.tsv", sep='\t')

In [ ]:
# Function to extract features considering weights as frequencies
def extract_weighted_features(df, num_bins=10):
    feature_df = pd.DataFrame()
    
    for patient_id, group in df.groupby('patient_id'):
        distances = group['distance']
        weights = group['weight']
        
        # Calculate weighted histogram
        hist, _ = np.histogram(distances, bins=num_bins, range=(distances.min(), distances.max()), weights=weights, density=True)
        
        # Calculate weighted statistics
        weighted_mean = np.average(distances, weights=weights)
        weighted_std = np.sqrt(np.average((distances - weighted_mean)**2, weights=weights))
        
        # Concatenate histogram and statistics as features
        features = np.concatenate([
            hist, 
            [weighted_mean, weighted_std, skew(distances), kurtosis(distances)]
        ])
        
        feature_df = pd.concat([feature_df, pd.Series(features, name=patient_id).to_frame().T])    
    
    return feature_df

In [ ]:
# Extract features
features = extract_weighted_features(combined_df)
y = combined_df.groupby('patient_id')['label'].first()

# Create explicit integer indices for features
feature_names = [f'feature_{i}' for i in range(len(features.columns))]

# Encode labels for class names
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.2, random_state=42)

# Optionally scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train Random Forest Classifier
# clf = RandomForestClassifier(bootstrap = False, random_state=42)
clf = RandomForestClassifier(random_state=42)

clf.fit(X_train_scaled, y_train)

# Predictions
y_pred = clf.predict(X_test_scaled)

# Evaluation
print(classification_report(y_test, y_pred))

# Create explicit class names
class_names = ['Healthy', 'Lupus']

In [ ]:
# Create more descriptive feature names
descriptive_feature_names = [
    'Hist_bin_1', 'Hist_bin_2', 'Hist_bin_3', 'Hist_bin_4', 'Hist_bin_5',
    'Hist_bin_6', 'Hist_bin_7', 'Hist_bin_8', 'Hist_bin_9', 'Hist_bin_10',
    'Weighted_Mean', 'Weighted_StdDev', 'Skewness', 'Kurtosis'
]

# Get feature importances
importances = clf.feature_importances_
std = np.std([tree.feature_importances_ for tree in clf.estimators_], axis=0)
indices = np.argsort(importances)[::-1]

# Plot the feature importances with descriptive names
plt.figure(figsize=(12, 6))
plt.title("Feature Importances in Random Forest", fontsize=14)
plt.bar(range(len(importances)), importances[indices], yerr=std[indices], align="center")
plt.xticks(range(len(importances)), [descriptive_feature_names[i] for i in indices], rotation=45, ha='right')
plt.ylabel("Importance", fontsize=12)
plt.tight_layout()
plt.show()

# Print feature importances with descriptive names
print("Feature ranking:")
for i in range(len(importances)):
    print(f"{i+1}. {descriptive_feature_names[indices[i]]}: {importances[indices[i]]:.4f}")

In [ ]:
# Define a function to visualize the trees
def visualize_tree(tree_index, classifier, feature_names, class_names):
    estimator = classifier.estimators_[tree_index]
    dot_data = export_graphviz(estimator, out_file=None,
                               feature_names=feature_names,
                               class_names=class_names,
                               filled=True, rounded=True,
                               special_characters=True)
    graph = pydotplus.graph_from_dot_data(dot_data)
    return Image(graph.create_png())

# How many trees to visualize
num_trees_to_show = min(3, len(clf.estimators_))

# Visualize multiple trees
for i in range(num_trees_to_show):
    print(f"Decision Tree {i+1}")
    display(visualize_tree(i, clf, feature_names, class_names))